Binary Logistic Regression
- A situation where the variable you are trying to predict is binary (1/0). So it can be either 0 or 1. for example, if you have a neural network that tries to predict whether there is cat in a picture, there will be an output neuron that is very close to 1 when cat, and close to 0 when not cat.
- in a neural network this is represented by a single neuron. 1 for present feature, 0 for target feature not present. But can have multiple of these neurons in the output layer. For example, one neuron could be person, not person and the other could be indoors/outdoors.
- uses sigmoid activation function rather than softmax, and binary cross entropy rather than categorical

Sigmoid Activation Function
- sigma(x) = 1 / (1 + e^-x)
- squishes the range of possible input values between 0 and 1, so very negative values are very close to 0, and very positive values are close to 1
- Derivative of sigmoid function => will be written in terms of the sigmoid function, similar to how we did softmax derivative
1) re-write = d/dz (1/(1 + e^-z)) = (1 + e^-z)^-1
2) use chain rule: -1 * ( 1+ e^-z) ^ -2 * d/dz (1 + e^-z) = -1 * ( 1+ e^-z) ^ -2 * (0 + e^-z * -1)
3) simplify, cancel negatives = ( 1+ e^-z) ^ -2 * e^-z
4) algebra: 1/ (1 + e^-z)^2 * e^-z = 1/ (1 + e^-z) * e^-z / (1 + e^-z)  = 1/ (1 + e^-z) * (1+ e^-z - 1) / (1 + e^-z) = 1/ (1 + e^-z) * ((1+ e^-z)/ (1 + e^-z) - 1/(1 + e^-z)) = 1/ (1 + e^-z) * (1 - 1/(1 + e^-z)) = sigma(x) * (1 - sigma(x))
- remeber sigmoid function is equal to 1 / (1 + e^-x), so we are just substituting it in above


Sigmoid object  
- we save output so we can just re use it in the backward pass.

In [ ]:
import numpy as np
# Sigmoid activation
class Activation_Sigmoid:
    # Forward pass
    def forward(self, inputs):
        # Save input and calculate/save output
        # of the sigmoid function
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))
    # Backward pass
    def backward(self, dvalues):
        # Derivative - calculates from output of the sigmoid function
        self.dinputs = dvalues * (1 - self.output) * self.output